In [1]:
!pip install git+git://github.com/cytomining/pycytominer@e43be528d3ca6d77d2b1a347fe8e4131dfc65bf0

  Cloning git://github.com/cytomining/pycytominer (to revision e43be528d3ca6d77d2b1a347fe8e4131dfc65bf0) to /tmp/pip-req-build-ni3ueef1
  Running command git clone --filter=blob:none --quiet git://github.com/cytomining/pycytominer /tmp/pip-req-build-ni3ueef1
  fatal: unable to connect to github.com:
  github.com[0: 140.82.121.3]: errno=Connection timed out

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet git://github.com/cytomining/pycytominer /tmp/pip-req-build-ni3ueef1 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet git://github.com/cytomining/pycytominer /tmp/pip-req-build-ni3ueef1 did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os

In [4]:
sys.path.append(os.path.abspath('../..'))

In [5]:
import json
import random
import pandas as pd

In [23]:
# taken from https://github.com/broadinstitute/lincs-profiling-complementarity/blob/master/1.Data-exploration/Consensus/cell_painting/1.cellpainting_moa_median_scores_calculation.ipynb

from pycytominer import feature_select

def feature_selection(data):
    """
    Perform feature selection by dropping columns with null or 
    only zeros values, and highly correlated values from the data.
    
    params: 
    dataset_link: string of github link to the consensus dataset

    Returns:
    data: returned consensus dataframe
    
    """
    # data = pd.read_csv(dataset_link, compression='gzip', error_bad_lines=False)
    cols = data.columns.tolist()
    drop_cols = [x for x in cols if ((data[x].isnull().sum()) | all(y == 0.0 for y in data[x].values))]
    data.drop(drop_cols, axis = 1, inplace = True)
    excluded_cols = feature_select(
        data,
        operation=[
            "correlation_threshold", # Exclude features that have correlations above a certain threshold (default 0.9)
            "variance_threshold" # Exclude features that have low variance (low information content)
        ],
        # blocklist_file="https://raw.githubusercontent.com/broadinstitute/lincs-cell-painting/1769b32c7cef3385ccc4cea7057386e8a1bde39a/utils/consensus_blocklist.txt"
    )
    return {'all_zero_or_nan': drop_cols, **excluded_cols}

In [ ]:
df=pd.read_csv('/sise/assafzar-group/g-and-n/plates/csvs/24278.csv', index_col=(list(range(6))))
df.shape

In [ ]:
exclude_cols = feature_selection(df)

In [ ]:
all_excols = set()
for t, l in ex_cols.items():
    print(t, len(l))
    all_excols |= set(l)
    
print('ALL', len(all_excols))

In [14]:
import numpy as np
import pandas as pd

In [24]:
# df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
df = pd.DataFrame(np.zeros((15,5)), columns=[f'Cells_Granularity_{i}_RNA' for i in range(1,6)])
df

,Cells_Granularity_1_RNA,Cells_Granularity_2_RNA,Cells_Granularity_3_RNA,Cells_Granularity_4_RNA,Cells_Granularity_5_RNA
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0


In [32]:
plates_pth = '/sise/assafzar-group/g-and-n/plates/csvs'
plates = os.listdir(plates_pth)
len(plates)

406

In [47]:
import random
plates.pop(random.randrange(len(plates)))

'25704.csv'

In [ ]:
processed = []
cur_df = None
while plates:
    cur_plate = plates.pop(random.randrange(len(plates)))
    processed.append(cur_plate)
    
    df = pd.read_csv(os.path.join(plates_pth, cur_plate), index_col=(list(range(6))))
    if cur_df is None:
        cur_df = df
    else:
        cur_df = pd.concat([cur_df, df])
    
    del df
        
    cols = feature_selection(cur_df)
    with open(f'/sise/assafzar-group/g-and-n/feature_selection/{len(processed)}.sav', 'w') as f:
        out = {'plates': processed, 'cols': cols}
        json.dump(out, f)
        del cols

In [ ]:
import os
job_cancel_str="scancel " + os.environ['SLURM_JOBID']
os.system(job_cancel_str)

# Load Feature Selection Output

In [6]:
cols_pth = '/storage/users/g-and-n/plates/columns.txt'
cols = json.load(open(cols_pth, 'r'))
chans = [c for c in cols.keys() if c not in ['MIXED']]
chans

# del cols['GENERAL']
del cols['MIXED']

cols['ALL'] = sum(cols.values(), [])

In [9]:
ds = {}
for c in chans:
    with open(f'/sise/assafzar-group/g-and-n/feature_selection/{c}_23.sav', 'r') as f:
        ds[c] = json.load(f)

In [31]:
for c, d in ds.items():
    print(c)
    for k, v in d['cols'].items():
        print('\t', k, len(v))

    all_cols = sum(d['cols'].values(), [])
    # all_cols = d['cols']['correlation_threshold'] + d['cols']['variance_threshold']
    
    
    c_uni = len(set(all_cols))
    c_all = len(cols[c])
    print('\t\t\t\t', 'Summary:', f'{c_uni}/{c_all}:={c_uni/c_all:.0%} ({c_all-c_uni})')

GENERAL
	 has_nulls 18
	 all_zeros 0
	 correlation_threshold 16
	 variance_threshold 25
				 Summary: 34/67:=51% (33)
AGP
	 has_nulls 24
	 all_zeros 0
	 correlation_threshold 34
	 variance_threshold 24
				 Summary: 62/86:=72% (24)
DNA
	 has_nulls 8
	 all_zeros 0
	 correlation_threshold 34
	 variance_threshold 4
				 Summary: 42/70:=60% (28)
ER
	 has_nulls 24
	 all_zeros 0
	 correlation_threshold 34
	 variance_threshold 19
				 Summary: 59/86:=69% (27)
Mito
	 has_nulls 24
	 all_zeros 0
	 correlation_threshold 33
	 variance_threshold 21
				 Summary: 59/86:=69% (27)
RNA
	 has_nulls 24
	 all_zeros 0
	 correlation_threshold 33
	 variance_threshold 19
				 Summary: 57/86:=66% (29)


## Check on random plate

In [49]:
for _ in range(10):
    cur_plate = plates.pop(random.randrange(len(plates)))
    print(cur_plate)

    df = pd.read_csv(os.path.join(plates_pth, cur_plate), index_col=(list(range(6))))

    for c, d in ds.items():
        print(c, df[d['cols']['has_nulls']].isna().sum().unique())

24684.csv
GENERAL [0]
AGP [0]
DNA [0]
ER [0]
Mito [0]
RNA [0]
26675.csv
GENERAL [0]
AGP [0]
DNA [0]
ER [0]
Mito [0]
RNA [0]
24564.csv
GENERAL [10  0]
AGP [10]
DNA [10]
ER [10]
Mito [10]
RNA [10]
24301.csv
GENERAL [10  0]
AGP [10]
DNA [10]
ER [10]
Mito [10]
RNA [10]
26204.csv
GENERAL [2 0]
AGP [2]
DNA [2]
ER [2]
Mito [2]
RNA [2]
26662.csv
GENERAL [1 0]
AGP [1]
DNA [1]
ER [1]
Mito [1]
RNA [1]
25694.csv
GENERAL [1 0]
AGP [1]
DNA [1]
ER [1]
Mito [1]
RNA [1]
24294.csv
GENERAL [8 0 1]
AGP [8]
DNA [8]
ER [8]
Mito [8]
RNA [8]
25885.csv
GENERAL [0]
AGP [0]
DNA [0]
ER [0]
Mito [0]
RNA [0]
25908.csv
GENERAL [1 0]
AGP [1]
DNA [1]
ER [1]
Mito [1]
RNA [1]


# Create new columns file

## FS per Channel

In [16]:
# del cols['GENERAL']
del cols['ALL']

In [68]:
cols.keys()

dict_keys(['GENERAL', 'AGP', 'DNA', 'ER', 'Mito', 'RNA'])

In [69]:
for c, d in ds.items():
    drop_cols = set(d['cols']['correlation_threshold'] + d['cols']['variance_threshold'])
    if c in cols.keys():
        cols[c] = [cl for cl in cols[c] if cl not in drop_cols]

In [70]:
for k, v in cols.items():
    print(k, len(v))

GENERAL 39
AGP 31
DNA 34
ER 35
Mito 34
RNA 35


In [71]:
with open('/storage/users/g-and-n/plates/columns-fs.json', 'w') as f:
    json.dump(cols, f)

## FS all channels

In [8]:
with open(f'/sise/assafzar-group/g-and-n/feature_selection/ALL_24.sav', 'r') as f:
    d_cols = json.load(f)

In [20]:
drop_cols = set(d_cols['cols']['correlation_threshold'] + d_cols['cols']['variance_threshold'])
for c, c_cols in cols.items():
    new_cols = [cl for cl in c_cols if cl not in drop_cols]
    print(c, len(new_cols), len(c_cols))
    cols[c] = new_cols

GENERAL 67 67
AGP 33 86
DNA 33 70
ER 31 86
Mito 31 86
RNA 28 86


In [22]:
with open('/storage/users/g-and-n/plates/columns-all-fs.json', 'w') as f:
    json.dump(cols, f)